# **Finding Lane Lines on the Road** 
***
**Context:** In this project, we'll learn to identify lane lines on the road.  We'll practice first on a single image, and later apply our tools to a video stream (really just a series of images).  Follow the steps below to complete the exercise.  Each step will be preceded by some "Context", and occasionally followed by a "Note" to the user.  

**Step 1:** Let's have a look at our first image called 'test.jpg'.  Run the next 2 cells below (hit Shift-Enter or the "play" button above) to bring up an interactive matplotlib window displaying the image.

**Note:** you can run each cell by first selecting it, then hitting "Shift-Enter" or pressing the play button in the toolbar above.  If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".

---

In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib qt

In [2]:
image = mpimg.imread('test.jpg')
print('This image is: ',type(image), 'with dimesions:', image.shape)
plt.imshow(image)
fig = plt.gcf()
fig.canvas.manager.window.raise_()

('This image is: ', <type 'numpy.ndarray'>, 'with dimesions:', (540, 960, 3))


___
**Context:** This is an image from a front facing camera on a car driving down the highway.  Have a look at the image. With our eyes, we can easily identify the lane lines right?

**Step 2:**  So let's brainstorm ways we might be able to identify them with an algorithm... what kind of features do you think we could use to identify the lane lines in the image? Write your ideas in the cell below:
___

**Double click here to enter your ideas:** 

---
**Context:** To start with, we're going to use color to identify the lane lines.  In the lesson we did this by hand, but here we're going to use an OpenCV function called "inRange()".   

We have our image stored in the variable called 'image', which is an array with shape = (540, 960, 3).  Thus, our image is 540 pixels high, 960 pixels wide, and contains 3 color layers, one each for Red, Green, and Blue (RGB).  

This is an 8 bit color image, meaning that values for [R,G,B] range from 0 to 255.  Pure white in an 8 bit image has [R,G,B] values of [255, 255, 255]. Our lane lines in the image aren't pure white, but they're close.  By implementing lower threshold (i.e., requiring RGB to be greater than some value) in [R,G,B] space, we can isolate pixels that are "whiteish".  


**Step 3:** Use the interactive matplotlib window to explore the colors in the image.  When you move your cursor over the image, you will see the values for the x and y position of the cursor, as well as the three value color vector displayed at the bottom of the window.  For example: x=885.952, y=118.532 [134,194,231], implies that at position [x, y] = [885.952, 118.532], the color values [Red, Green, Blue] = [134, 194, 231].  

Use the zoom tool (magnifying glass icon) in the matplotlib window toolbar to zoom in on the lane lines and explore their color.   Determine an appropriate lower threshold in [R,G,B] space for identifying pixels that are the same color as the lines.  

In the cell below, replace the values in the variable "lowWhiteThreshold"  with your threshold values for [R,G,B].  We will then use the *cv2.inRange()* function to identify pixels that meet our color criteria.  This function returns a binary image with just one layer of color information, where all pixels meeting the criteria have been set to 255, and all others to 0. 

Run the cell below and try to set a threshold that retains the maximum number of pixels that correspond to lines on the road, without including a whole lot of pixels that are not the lines. Tweak your "lowWhiteThreshold" values until you get a result that looks approximately like this example image below. 

**Note:** hit the "home" icon in the matplotlib window to refresh and see the full image. 

---

In [3]:
def colorSelect(image, white_low, white_high):
    lower_white = np.array(white_low, dtype=np.uint8) 
    upper_white = np.array(white_high, dtype=np.uint8) 
    selected = cv2.inRange(image, lower_white, upper_white)
    return selected

lowWhiteThreshold = [1, 1, 1]
highWhiteThreshold = [255, 255, 255]
cselect = colorSelect(image, lowWhiteThreshold, highWhiteThreshold)
#cv2.imwrite('color_selected.jpg', cselect)
plt.imshow(cselect, cmap='Greys_r')
fig = plt.gcf()
fig.canvas.manager.window.raise_()

<figure>
 <img src="color_selected.jpg" width="480" alt="Color Selected Image" />
 <p/>
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output image should look approximately like this after color selection </p> 
 </figcaption>
</figure>

---

**Context:** We have now drastically simplified our image to contain only binary values (0 or 255), where zeros correspond to pixels that did not meet our color criterion.  

We are now ready to detect lines in the image using a "Hough Transform". A Hough transform can be used to detect various shapes in an image, but here we'll
use it to detect lines.  Check out the <A HREF="http://docs.opencv.org/2.4/modules/imgproc/doc/feature_detection.html#HoughLinesP" target="_blank"> OpenCV docs on feature detection</A> and scroll down to the HoughLinesP() function learn more.

**Step 4:** Here we'll write another couple functions to perform the Hough transform and then draw the detected lines back onto our original image. The Hough Transform takes several parameters (rho, theta, threshold, minLineLength, and maxLineGap).  Read the docs in the links above to find out what these parameters do, and play around with different values until your output image looks roughly like the sample image below.

**Note:** cv2.HoughLinesP() returns "lines", which is an array containing the endpoints of all the line segments detected.  So, for example, lines[0] will be a four element array [x1, y1, x2, y2], where (x1, y1) and (x2, y2) are the endpoints of the first detected line segment.

After detecting lines, we define a function "drawLane()" to draw each of the detected line segments onto our image.

---

In [4]:
def houghLinesP(imageCopy, cselected, rho, theta, threshold, minLineLength, maxLineGap):
    lines = cv2.HoughLinesP(cselected, rho, theta, threshold, np.array([]),
                            minLineLength, maxLineGap)
    drawLane(imageCopy, lines)
    return lines, imageCopy

def drawLane(image, lines):
    if lines is not None:
        for line in lines:
            for x1,y1,x2,y2 in line:
                 cv2.line(image,(x1,y1),(x2,y2),(200,0,0),2)
    return

#we'll actually draw the lines onto a blank image, then coadd it to the original below
def blank(image):
    return np.zeros(image.shape, np.uint8)

#the Hough transform takes various parameters
#see the links above to learn more about what they do.
rho = 1
theta = np.pi/180
threshold = 1
minLineLength = 10
maxLineGap = 3

lines, lineImage = houghLinesP(blank(image), cselect, rho, theta, threshold, minLineLength, maxLineGap)
combo = cv2.addWeighted(image, 0.8, lineImage, 1, 0)

#cv2.imwrite('laneLines_firstPass.jpg', cv2.cvtColor(combo, cv2.COLOR_BGR2RGB))
plt.imshow(combo)
fig = plt.gcf()
fig.canvas.manager.window.raise_()

<figure>
 <img src="laneLines_firstPass.jpg" width="480" alt="Combined Image" />
 <p/>
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output image should look approximately like this after line detection </p> 
 </figcaption>
</figure>


---
**Context:** At this point, we have detected many lines in our color selected image.  Clearly, some of them correspond to our lane lines, and others correspond to junk we don't want.  To get rid of the junk, we will define a "region of interest" in the image, such that, going forward, we will only try to detect lines in this region.

 In this step, we want to isolate the region in the image where as many of the line segments that we want are, and eliminate line segments that we don't want.  While we could make it any shape we want, we will start by defining a triangular region that basically includes our driving lane and nothing else.
 
**Step 5:** Choose values for the vertices of a triangle (modify the array called "vertices" below) that encompasses our region of interest in the image. Run the cell below and examine the output.  Your output image should look like the masked image shown below.

**Note:** The y-axis in the image is flipped, so the values run from 0 at the top of the frame to 539 at the bottom.

---

In [5]:
def regionOfInterest(img, verts):
    
    mask = blank(img)
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,)*channel_count
    else:
        ignore_mask_color = 255
        
    cv2.fillPoly(mask, verts, ignore_mask_color)
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


vertices = np.array([[(0,539), (959,400), (440,0)]], dtype=np.int32)
masked = regionOfInterest(image, vertices)
#cv2.imwrite('masked.jpg', cv2.cvtColor(masked, cv2.COLOR_BGR2RGB))

plt.imshow(masked)
fig = plt.gcf()
fig.canvas.manager.window.raise_()

<figure>
 <img src="masked.jpg" width="480" alt="Combined Image" />
 <p/>
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output image should look approximately like this after masking </p> 
 </figcaption>
</figure>


---
**Context:** Now we have identified our region of interest.  We will isolate this region of interest in our color selected image from above, and rerun our line detection.
 
**Step 6:** Run the cell below to create a masked version of the color selected image and rerun the line detection step.

---

In [6]:
masked = regionOfInterest(cselect, vertices)
lines, lineImage = houghLinesP(blank(image), masked, rho, theta, threshold, minLineLength, maxLineGap)
combo = cv2.addWeighted(image, 0.75, lineImage, 1, 0)
#cv2.imwrite('laneLines_secondPass.jpg', cv2.cvtColor(combo, cv2.COLOR_BGR2RGB))
plt.imshow(combo)
fig = plt.gcf()
fig.canvas.manager.window.raise_()

<figure>
 <img src="laneLines_secondPass.jpg" width="480" alt="Combined Image" />
 <p/>
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output image should look approximately like this after line detection </p> 
 </figcaption>
</figure>


---
**Context:** Now we've got our lane lines identified, and no extraneous junk to worry about. We'd now like to draw an estimate of where the lane is on the image, based on the detected line segments.  To do this, we will fit a line to to all the detected line segments, and extrapolate up and down the road. 

At this point, we're also going to take into account the slope of the detected line segments, taking advantage of the fact that we know our line detections for the righthand lane line should all have positive slope, and those for the line on the left have negative slope (recall that the y-axis is flipped).
 
**Step 7:** Run the cell below to redefine the drawLane function, now to extrapolate the detected line segments up and down the road.  This will output the original image with our full lane lines drawn on.

---

In [7]:
def houghLinesPExtra(imageCopy, cselected, rho, theta, threshold, minLineLength, maxLineGap):
    lines = cv2.HoughLinesP(cselected, rho, theta, threshold, np.array([]),
                            minLineLength, maxLineGap)
    drawLaneExtra(imageCopy, lines)
    return lines, imageCopy


def drawLaneExtra(image, lines):
    
    midpointx = vertices[0][2][0]
    rightX = []
    leftX = []
    rightY = []
    leftY = []
    slopeCut = 0.6  
    if lines is not None:
        for line in lines:
            for x1,y1,x2,y2 in line:
                rise = float(y2 - y1)
                run = float(x2 - x1)
                if run != 0:
                    slope = rise/run  
                    if slope > slopeCut and x1 > (midpointx-20) and x2 > (midpointx-20): 
                        #identifying lane line segments on the right side of the frame
                        rightX.extend([x1, x2])
                        rightY.extend([y1, y2])
                    elif slope < -slopeCut and x1 < (midpointx+20) and x2 < (midpointx+20): 
                        #identifying lane line segments on the left side of the frame
                        leftX.extend([x1, x2])
                        leftY.extend([y1, y2])                   

        if leftX and leftY:
            fitLeft = np.polyfit(leftX, leftY, 1)
            startYleft = 0
            endYleft = image.shape[0]
            startXleft = int((startYleft - fitLeft[1]) / fitLeft[0])
            endXleft = int((endYleft - fitLeft[1]) / fitLeft[0])
            cv2.line(image,(startXleft,startYleft),(endXleft,endYleft),(255,0,0),10)
            
        if rightX and rightY:
            fitRight = np.polyfit(rightX, rightY, 1)
            startYright = 0
            endYright = image.shape[0]
            startXright = int((startYright - fitRight[1]) / fitRight[0])
            endXright = int((endYright - fitRight[1]) / fitRight[0])
            cv2.line(image,(startXright,startYright),(endXright,endYright),(255,0,0),10)
        
lines, lineImage = houghLinesPExtra(blank(image), masked, rho, theta, 
                                    threshold, minLineLength, maxLineGap)
lineMasked = regionOfInterest(lineImage, vertices)
combo = cv2.addWeighted(image, 0.75, lineMasked, 1, 0)
#cv2.imwrite('laneLines_thirdPass.jpg', cv2.cvtColor(combo, cv2.COLOR_BGR2RGB))
plt.imshow(combo)
fig = plt.gcf()
fig.canvas.manager.window.raise_()

<figure>
 <img src="laneLines_thirdPass.jpg" width="480" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output image should look approximately like this after line extrapolation </p> 
 </figcaption>
</figure>


---
**Context:** We've done it!  We now have a pretty good estimate of where the lane is.  
 
**Step 8:** In the cell below, explain in your own words what steps we took to get to this point.

---

**Double click here to enter your explanation:**

---
**Context:** We are now ready to run our code on a video stream.
 
**Step 9:** Run the cell below to see how our algorithm does on a stream of images.

**Note:** The cv2 display window may pop up behind your browser.

---

In [8]:
cv2.startWindowThread()
cv2.namedWindow('Lane-Finding')
cap = cv2.VideoCapture('Highway_Driving.mp4')
count = 0
while cap.isOpened():
    count += 1
    ret, image = cap.read()
    if image is not None:
        cselect = colorSelect(image, lowWhiteThreshold, highWhiteThreshold)
        masked = regionOfInterest(cselect, vertices)
        lines, lineImage = houghLinesPExtra(blank(image), masked, rho, theta, 
                                            threshold, minLineLength, maxLineGap)
        lineMasked = regionOfInterest(lineImage, vertices)
        combo = cv2.addWeighted(image, 0.75, cv2.cvtColor(lineMasked, cv2.COLOR_BGR2RGB), 1, 0)
        cv2.imshow('Lane-Finding',combo)
    else:
        break
        
    k = cv2.waitKey(30) & 0xff
    if k == 27 :
        break

cap.release()
cv2.destroyAllWindows()

---
**Context:** Fantastic!  It works (more or less)!  Things to think about going forward are: Where is this going to fail?  How can we make our algorithm more robust? 
 
**Step 10:** In the cell below, write down your thoughts on possible failure modes and how to improve our algorithm:

---

**Double click here to enter your ideas:**

---
**Context:** Let's test a new scenario and see how our algorithm performs.
<p></p> 
<p></p> 
<figure>
 <img src="test2.jpg" width="480" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> What will we find using this new image? </p> 
 </figcaption>
</figure>
 <p></p> 
**Step 11:** Run the cell below to test our algorithm on a new image.

---

In [9]:
image = mpimg.imread('test2.jpg')

cselect = colorSelect(image, lowWhiteThreshold, highWhiteThreshold)
vertices = vertices #might want to change this!
masked = regionOfInterest(image, vertices)

plt.imshow(masked)
fig = plt.gcf()
fig.canvas.manager.window.raise_()

In [10]:
masked = regionOfInterest(cselect, vertices)
lines, lineImage = houghLinesPExtra(blank(image), masked, rho, theta, 
                                    threshold, minLineLength, maxLineGap)
lineMasked = regionOfInterest(lineImage, vertices)
combo = cv2.addWeighted(image, 0.75, lineMasked, 1, 0)
plt.imshow(combo)#, cmap = 'Greys_r')
fig = plt.gcf()
fig.canvas.manager.window.raise_()

---
**Context:** It seems we have a problem here!  What do you think is going wrong?

**Step 12:** In the cell below, write down your thoughts on what the problem is and how to accommodate this issue in our algorithm:

---

**Double click here to enter your ideas:**

**Context:** As we well know, not all lane lines are white, so we'll have to take this into account.  At this point, we'll shift away from a color selection, and try to improve our detection of the lines solely based on shape.

In this next series of cells, we will perform edge detection on our input image.  First, however, we need to convert to grayscale (i.e. go from 3 dimensions in color to just 1).  

**Step 12:** Run the cell below to convert our input image to grayscale and display it.

In [11]:
gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
print('Grayscale image is: ',type(gray), 'with dimesions:', gray.shape)
#cv2.imwrite('grayScale.jpg', gray)
plt.imshow(gray, cmap='Greys_r')
fig = plt.gcf()
fig.canvas.manager.window.raise_()

('Grayscale image is: ', <type 'numpy.ndarray'>, 'with dimesions:', (540, 960))


<figure>
 <img src="grayScale.jpg" width="480" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output image should look like this after grayscaling </p> 
 </figcaption>
</figure>

**Context:** We can see from the printout that our grayscale image is two dimensional, reduced from three dimensions in the original color image.

We see a problem however, in that the yellow line on the left has been converted into a medium shade of gray, which will be hard to detect against the background.

At this point, we back up a step and try a color transform known as "Hue, Light, and Saturation" (HLS) before grayscaling the image.  This conversion is commonly used to bring out the brighter colors in an image.  After the conversion we'll grayscale by simply summing along the color axis.

**Step 12:** Run the cell below to convert our input image to grayscale via HLS color transform and display it.

In [12]:
def grayViaHLS(img):
    hls = cv2.cvtColor(img,cv2.COLOR_BGR2HLS)
    hlsGray = np.uint8(np.sum(hls, axis=2)/3)
    return hlsGray

hlsGray = grayViaHLS(image)
#cv2.imwrite('hlsGray.jpg', hlsGray)
plt.imshow(hlsGray, cmap='Greys_r')
fig = plt.gcf()
fig.canvas.manager.window.raise_()

<figure>
 <img src="hlsGray.jpg" width="480" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output image should look like this after grayscaling via HLS </p> 
 </figcaption>
</figure>

**Context:** Great!  We've got our yellow line back and now we're ready to detect edges.  To do this we'll use a Canny edge detector (an algorithm developed by John F. Canny in 1986).

The basic premise behind the Canny edge detection algorithm is that we're going to look for rapid changes in brightness from pixel to pixel across our image.  You'll first smooth the image with a Gaussian kernel to get rid of noise fluctuations that might otherwise look like edges to the Canny algorithm.  You'll then measure the derivative (change) of intensity from pixel to pixel, both vertically and horizontally, at all points in the image.  Then play with the thresholds to appropriately to pick out the edges you want.  Check out <A HREF="http://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_canny/py_canny.html" target="_blank"> the OpenCV docs on Canny edge detection</A> to learn more

**Step 13:** Run the cell below and play with the parameters (kernelSize, lowerThresh, upperThresh, and apSize) to perform Canny edge detection on our grayscaled image.  The output will be a binary image showing all pixels associated with edges in white.

In [13]:
kernelSize = 5
blurGray = cv2.GaussianBlur(hlsGray,(kernelSize, kernelSize),0)

lowerThresh = 50
upperThresh = 150
apSize = 3
edges = cv2.Canny(blurGray, lowerThresh, upperThresh, apertureSize = apSize)
#cv2.imwrite('edges.jpg', edges)
plt.imshow(edges, cmap='Greys_r')
fig = plt.gcf()
fig.canvas.manager.window.raise_()

<figure>
 <img src="edges.jpg" width="480" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output image should look like this after edge detection </p> 
 </figcaption>
</figure>

**Context:** Ok, now we've got our edges detected and we can run our Hough transform on the output of the Canny algorithm to detect lines.  First, we'll crop out our region of interest as before.

**Step 13:** Run the cell below to crop the edge detected image and run a Hough transform on it to detect lines.  We'll then plot up the detected lines on the original image.

In [14]:
masked = regionOfInterest(edges, vertices)
lines, lineImage = houghLinesPExtra(blank(image), masked, rho, theta, 
                                    threshold, minLineLength, maxLineGap)
lineMasked = regionOfInterest(lineImage, vertices)
combo = cv2.addWeighted(image, 0.75, lineMasked, 1, 0)
#cv2.imwrite('laneLines_fourthPass.jpg', cv2.cvtColor(combo, cv2.COLOR_BGR2RGB))
plt.imshow(combo)#, cmap = 'Greys_r')
fig = plt.gcf()
fig.canvas.manager.window.raise_()

<figure>
 <img src="laneLines_fourthPass.jpg" width="480" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output image should look like this after line detection </p> 
 </figcaption>
</figure>

**Context:** Nailed it!  Awesome!  Now lets try another video stream

**Step 13:** Run the cell below to apply our new algorithm to a video stream.  

In [15]:
cv2.startWindowThread()
cv2.namedWindow('Lane-Finding')
cap = cv2.VideoCapture('yellowLeft.mp4')

while cap.isOpened():

    ret, image = cap.read()
    if image is not None:
        hlsGray = grayViaHLS(image)
        blurGray = cv2.GaussianBlur(hlsGray,(kernelSize, kernelSize), 0)
        edges = cv2.Canny(blurGray, lowerThresh, upperThresh, apertureSize = apSize)
        masked = regionOfInterest(edges, vertices)
        lines, lineImage = houghLinesPExtra(blank(image), masked, rho, theta, 
                                            threshold, minLineLength, maxLineGap)
        lineMasked = regionOfInterest(lineImage, vertices)
        combo = cv2.addWeighted(image, 0.75, cv2.cvtColor(lineMasked, cv2.COLOR_BGR2RGB), 1, 0)
        cv2.imshow('Lane-Finding',combo)
    else:
        break
        
    k = cv2.waitKey(30) & 0xff
    if k == 27 :
        break

cap.release()
cv2.destroyAllWindows()